In [1]:
import xml.etree.ElementTree as ET
import sqlite3
import os

# Creating Initial tests to see if it is converting xml to sql statements

In [2]:
#Create a mapping function that takes in different SQL data types 
MAPPING = {
    'str': 'VARCHAR2(70)',
    'int': 'NUMBER',
    'float': 'DECIMAL(10, 2)'
}

#Define an XML string that represents an employee data
xml = '''<EMPLOYEE_DETAILS>
  <EMPLOYEE>
    <EMP_ID>101</EMP_ID>
    <EMP_NAME>Marshall Walker</EMP_NAME>
    <EMP_SALARY>100000.00</EMP_SALARY>
    <EMP_DEPT>100</EMP_DEPT>
    <EMP_JOB_ID>IT001</EMP_JOB_ID>
  </EMPLOYEE>
</EMPLOYEE_DETAILS>'''

# Define a helper function to determine the data type of a value
def _get_type(data):
    #Initialize variables to track if the data is a float or an integer
    is_float = False
    is_int = False
    
    try:
        #Begin to convert data to float or an integer, and set the appropriate flags if successful
        float(data)
        is_float = True
        int(data)
        is_int = True
    except ValueError:
        # if the data cannot be converted to a float or an integer, then it should be string
        pass
    if is_int and is_float:
        # If data can be both a float and an int, then it is an int
        return 'int'
    elif is_float:
        #If the data is a float then it is a float
        return 'float'
    else:
        #If the data cannot be converted to a float or int, then it will be a string
        return 'str'

#Parse the XML string into an ELementTree object
root = ET.fromstring(xml)

#Find the Employee element and get the child elements 
emp_properties = root.find('.//EMPLOYEE')

#Print the beginning of the CREATE TABLE statement
print('CREATE TABLE BANK_EMPLOYEE(')

#Loop through each property of the employee element
for idx, prop in enumerate(emp_properties):
    #Loop through each of the employee element
    last_prop = idx == len(emp_properties) - 1
    #then check if its the last property
    if last_prop:
        #Print one with comma
        print(f'\t{prop.tag} {MAPPING[_get_type(prop.text)]}')
    else:
        #otherwise
        print(f'\t{prop.tag} {MAPPING[_get_type(prop.text)]},')

print(');')

#Create a list of values for the insert statement
values = []
for prop in emp_properties:
    values.append(prop.text)

#Print the beginning of the insert statement
print('INSERT INTO BANK_EMPLOYEES VALUES(')

#Loop through each value in the values list
for idx, value in enumerate(values):
    #Loop through each value
    last_value = idx == len(values) - 1
    
    #check if it is the last value
    if last_value:
        # Print the value without comma
        print(f'\t{value}')
    else:
        #Print the value with comma
        print(f'\t{value}')

#print the end of the insert statement
print(');')

CREATE TABLE BANK_EMPLOYEE(
	EMP_ID NUMBER,
	EMP_NAME VARCHAR2(70),
	EMP_SALARY DECIMAL(10, 2),
	EMP_DEPT NUMBER,
	EMP_JOB_ID VARCHAR2(70)
);
INSERT INTO BANK_EMPLOYEES VALUES(
	101
	Marshall Walker
	100000.00
	100
	IT001
);


# Creating a function that converts xml to sql statements by creating a table and inserting statements used in sql

In [5]:
def create_table_and_inserts(xml_file):
    #parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    #Get the column names and data types from the first row
    columns = []
    for child in root[0]:
        columns.append(child.tag)
    data_types = {}
    for child in root[0]:
        # Store the data type of each column by checking the type of the text
        data_types[child.tag] = type(child.text).__name__
        
#     #Generate the CREATE TABLE statement
#     create_table_sql = "CREATE TABLE {} (".format(root.tag) #start creatinf the sql statements to create a table with the name of the root tag
#     for column in columns: #Iterate through each column in the table
#         if data_types[column] == 'str': #Check if the data type of the column is a string
#             create_table_sql += "{} VARCHAR2, ".format(column) #Add the column name and data type (TEXT) to the SQL statement
#         elif data_types[column] == 'int': #Check if the data type of the column is an integer
#             create_table_sql += "{} INT".format(column) # Add the column name and data type (INT) to the SQL statement
#         elif data_types[column] == 'float': #Check if the data type of the column is a float
#             create_table_sql += "{} DECIMAL(10, 2), ".format(column) #Add the column name and data type (INT) to the SQL statement
#     create_table_sql = create_table_sql[:-2] + ");" # Remove the trailing comma and space, and add the closing parenthesis to the SQL statement
    
    # Define a dictionary that maps Python data types to SQL data types
    sql_data_types = {
        'str': 'VARCHAR2(70)',
        'int': 'NUMBER',
        'float': 'DECIMAL(10, 2)'
    }

    # Generate the CREATE TABLE statement
    create_table_sql = "CREATE TABLE {}(\n".format(root.tag)
    for column in columns:
        # Add each column and its data type to the CREATE TABLE statement
         #create_table_sql += "  {} {},\n".format(column, sql_data_types[data_types[column]])
        create_table_sql += f"  {column} {sql_data_types[_get_type(root[0].find(column).text)]},\n"
    # Remove the trailing comma and space and add a closing parenthesis to the CREATE TABLE statement
    create_table_sql = create_table_sql[:-2] + "\n);"


    #Generate the INSERT INTO statements
    insert_sqls =[]
    for row in root:
        #start building the INSERT statement with the table name
        insert_sql = "INSERT INTO {} (".format(root.tag)
        
        #Add the column names to the INSERT statement
        for column in columns:
            insert_sql += "{}, ".format(column)
        insert_sql = insert_sql[:-2] + ") VALUES (" # Remove the trailing comma and space
        for child in row:
            if data_types[child.tag] == 'str':
                insert_sql += "'{}', ".format(child.text) #Add the string value wrapped in quotes
            elif data_types[child.tag] == 'int':
                insert_sql += "{}, ".format(int(child.text)) #Add the integer value
            elif data_types[child.tag] == 'float':
                insert_sql += "{}, ".format(float(child.text)) # Add the float value
        insert_sql = insert_sql[:-2] + ");" #Remove the trailing comma and add closing parenthesis
        insert_sqls.append(insert_sql) #Add the complete INSERT statement to the list of insert_sqls

    return (create_table_sql, insert_sqls) #Return the CREATE TABLE statement and the list of INSERT statements

def _get_type(data):
    is_float = False
    is_int = False
    try:
        float(data)
        is_float = True
        int(data)
        is_int = True
    except ValueError:
        pass
    if is_int and is_float:
        return 'int'
    elif is_float:
        return 'float'
    else:
        return 'str'

In [6]:
xml_file = 'bank_employee.xml'
create_table_sql, insert_sqls = create_table_and_inserts(xml_file)
print(create_table_sql)
for insert_sql in insert_sqls:
    print(insert_sql)


CREATE TABLE EMPLOYEE_DETAILS(
  EMP_ID NUMBER,
  EMP_NAME VARCHAR2(70),
  EMP_SALARY DECIMAL(10, 2),
  EMP_DEPT NUMBER,
  EMP_JOB_ID VARCHAR2(70)
);
INSERT INTO EMPLOYEE_DETAILS (EMP_ID, EMP_NAME, EMP_SALARY, EMP_DEPT, EMP_JOB_ID) VALUES ('101', 'Marshall Walker', '100000.00', '100', 'IT001');
INSERT INTO EMPLOYEE_DETAILS (EMP_ID, EMP_NAME, EMP_SALARY, EMP_DEPT, EMP_JOB_ID) VALUES ('102', 'Arul Sharma', '250000.00', '105', 'CSE01');
INSERT INTO EMPLOYEE_DETAILS (EMP_ID, EMP_NAME, EMP_SALARY, EMP_DEPT, EMP_JOB_ID) VALUES ('103', 'Aaron Schmidt', '150000.00', '108', 'ECE08');
INSERT INTO EMPLOYEE_DETAILS (EMP_ID, EMP_NAME, EMP_SALARY, EMP_DEPT, EMP_JOB_ID) VALUES ('104', 'William Johansson', '225000.00', '109', 'ECE08');
INSERT INTO EMPLOYEE_DETAILS (EMP_ID, EMP_NAME, EMP_SALARY, EMP_DEPT, EMP_JOB_ID) VALUES ('105', 'Kate Young', '150000.00', '110', 'ECE10');


In [7]:
xml_file = 'bank_account.xml'
create_table_sql, insert_sqls = create_table_and_inserts(xml_file)
print(create_table_sql)
for insert_sql in insert_sqls:
    print(insert_sql)

CREATE TABLE BANK_DETAILS(
  ACCOUNT_NO NUMBER,
  CUSTOMER_ID NUMBER,
  HOLDER_NAME VARCHAR2(70),
  BALANCE DECIMAL(10, 2),
  ACCOUNT_TYPE VARCHAR2(70)
);
INSERT INTO BANK_DETAILS (ACCOUNT_NO, CUSTOMER_ID, HOLDER_NAME, BALANCE, ACCOUNT_TYPE) VALUES ('101', '110', 'John Smith', '5000.00', 'Savings');
INSERT INTO BANK_DETAILS (ACCOUNT_NO, CUSTOMER_ID, HOLDER_NAME, BALANCE, ACCOUNT_TYPE) VALUES ('102', '111', 'Jane Doe', '12000.00', 'Checking');
INSERT INTO BANK_DETAILS (ACCOUNT_NO, CUSTOMER_ID, HOLDER_NAME, BALANCE, ACCOUNT_TYPE) VALUES ('103', '112', 'Robert Johnson', '2000.00', 'Savings');
INSERT INTO BANK_DETAILS (ACCOUNT_NO, CUSTOMER_ID, HOLDER_NAME, BALANCE, ACCOUNT_TYPE) VALUES ('104', '113', 'Emily Davis', '15000.00', 'Checking');
INSERT INTO BANK_DETAILS (ACCOUNT_NO, CUSTOMER_ID, HOLDER_NAME, BALANCE, ACCOUNT_TYPE) VALUES ('105', '114', 'Michael Johnson', '7500.00', 'Savings');


# XML obtained from outside source

In [8]:
xml_file = 'fruits.xml'
create_table_sql, insert_sqls = create_table_and_inserts(xml_file)
print(create_table_sql)
for insert_sql in insert_sqls:
    print(insert_sql)

CREATE TABLE fruits(
  id NUMBER,
  name VARCHAR2(70),
  price NUMBER,
  quantity NUMBER
);
INSERT INTO fruits (id, name, price, quantity) VALUES ('1000', 'Apple', '4', '133');
INSERT INTO fruits (id, name, price, quantity) VALUES ('1001', 'Apricot', '5', '175');
INSERT INTO fruits (id, name, price, quantity) VALUES ('1002', 'Avocado', '5', '182');
INSERT INTO fruits (id, name, price, quantity) VALUES ('1003', 'Banana', '5', '187');
INSERT INTO fruits (id, name, price, quantity) VALUES ('1004', 'Bilberry', '5', '160');
INSERT INTO fruits (id, name, price, quantity) VALUES ('1005', 'Blackberry', '4', '178');
INSERT INTO fruits (id, name, price, quantity) VALUES ('1006', 'Blackcurrant', '5', '102');
INSERT INTO fruits (id, name, price, quantity) VALUES ('1007', 'Blueberry', '6', '156');
INSERT INTO fruits (id, name, price, quantity) VALUES ('1008', 'Currant', '5', '194');
INSERT INTO fruits (id, name, price, quantity) VALUES ('1009', 'Cherry', '5', '182');
INSERT INTO fruits (id, name, p

# Now test it by connecting to SQLLite server and generate 3 databases

In [9]:
def create_table_and_inserts(xml_files):
    # Connect to the database
    conn = sqlite3.connect('bank.db')
    cursor = conn.cursor()

    # Iterate over the XML files
    for xml_file in xml_files:
        # Parse the XML file
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # Get the column names and data types from the first row
        columns = []
        for child in root[0]:
            columns.append(child.tag)
        data_types = {}
        for child in root[0]:
            data_types[child.tag] = type(child.text).__name__

#         # Generate the CREATE TABLE statement
#         create_table_sql = "CREATE TABLE IF NOT EXISTS {} (".format(root.tag)
#         for column in columns:
#             if data_types[column] == 'str':
#                 create_table_sql += "{} TEXT, ".format(column)
#             elif data_types[column] == 'int':
#                 create_table_sql += "{} INTEGER, ".format(column)
#             elif data_types[column] == 'float':
#                 create_table_sql += "{} DECIMAL(10, 2), ".format(column)
#         create_table_sql = create_table_sql[:-2] + ");"

        sql_data_types = {
        'str': 'VARCHAR2(70)',
        'int': 'NUMBER',
        'float': 'DECIMAL(10, 2)'
        }

        # Generate the CREATE TABLE statement
        create_table_sql = "CREATE TABLE {}(\n".format(root.tag)
        for column in columns:
            create_table_sql += "  {} {},\n".format(column, sql_data_types[data_types[column]])
        create_table_sql = create_table_sql[:-2] + "\n);"
        cursor.execute(create_table_sql)

        # Generate the INSERT INTO statements and execute them
        for row in root:
            insert_sql = "INSERT INTO {} (".format(root.tag)
            for column in columns:
                insert_sql += "{}, ".format(column)
            insert_sql = insert_sql[:-2] + ") VALUES ("
            for child in row:
                if data_types[child.tag] == 'str':
                    insert_sql += "'{}', ".format(child.text)
                elif data_types[child.tag] == 'int':
                    insert_sql += "{}, ".format(int(child.text))
                elif data_types[child.tag] == 'float':
                    insert_sql += "{}, ".format(float(child.text))
            insert_sql = insert_sql[:-2] + ");"
            cursor.execute(insert_sql)

    # Commit changes and close the database connection
    conn.commit()
    conn.close()

# Call the function to create the tables and insert the data
xml_files = ['bank_employee.xml', 'bank_account.xml', 'fruits.xml']
create_table_and_inserts(xml_files)


# Now generalizing the code to create sender and receiver tables 

# Creating a sender table

In [ ]:
#Connect to the sender's database
sender_conn = sqlite3.connect('sender.db')
sender_cursor = sender_conn.cursor()

#create the sender table 
sender_cursor.execute('''CREATE TABLE IF NOT EXISTS xml_data (
                         filename TEXT,
                         xml TEXT
                        )''')

#create a list of XML files to send
xml_files = ['bank_employee.xml', 'bank_account.xml', 'fruits.xml']

#Loop through the XML files and insert the XML data into the sender database
for xml_file in xml_files:
    with open(xml_file, 'r') as f:
        xml_data = f.read()
    sender_cursor.execute('INSERT INTO xml_data (filename, xml) VALUES(?,?)', (xml_file, xml_data))
    sender_conn.commit()
    
sender_cursor.close()
sender_conn.close()

# Now creating a receiver table

In [ ]:
# Connect to the receiver database
receiver_conn = sqlite3.connect('receiver.db')
receiver_cursor = receiver_conn.cursor()

# Retrieve the XML data from the sender database
sender_conn = sqlite3.connect('sender.db')
sender_cursor = sender_conn.cursor()
sender_cursor.execute('SELECT filename, xml FROM xml_data')
xml_rows = sender_cursor.fetchall()
sender_cursor.close()
sender_conn.close()

# Loop through the XML rows and insert the data into the receiver database
for row in xml_rows:
    filename, xml_data = row

    # Parse the XML data and extract the column names and data types
    root = ET.fromstring(xml_data)
    columns = []
    data_types = {}
    for child in root[0]:
        columns.append(child.tag)
        data_types[child.tag] = type(child.text).__name__

#     create_table_sql = "CREATE TABLE IF NOT EXISTS {} (".format(root.tag)
#     for column in columns:
#         if data_types[column] == 'str':
#             create_table_sql += "{} TEXT, ".format(column)
#         elif data_types[column] == 'int':
#             create_table_sql += "{} INTEGER, ".format(column)
#         elif data_types[column] == 'float':
#             create_table_sql += "{} DECIMAL(10, 2), ".format(column)
#     create_table_sql = create_table_sql[:-2] + ");"
    # Define a dictionary that maps Python data types to SQL data types
    
    # Generate the CREATE TABLE statement
    sql_data_types = {
        'str': 'VARCHAR2(70)',
        'int': 'NUMBER',
        'float': 'DECIMAL(10, 2)'
    }

    # Generate the CREATE TABLE statement
    create_table_sql = "CREATE TABLE {}(\n".format(root.tag)
    for column in columns:
        # Add each column and its data type to the CREATE TABLE statement
        create_table_sql += "  {} {},\n".format(column, sql_data_types[data_types[column]])

    # Remove the trailing comma and space and add a closing parenthesis to the CREATE TABLE statement
    create_table_sql = create_table_sql[:-2] + "\n);"
    receiver_cursor.execute(create_table_sql)

    # Generate the INSERT INTO statements and execute them
    for row in root:
        insert_sql = "INSERT INTO {} (".format(root.tag)
        for column in columns:
            insert_sql += "{}, ".format(column)
        insert_sql = insert_sql[:-2] + ") VALUES ("
        for child in row:
            if data_types[child.tag] == 'str':
                insert_sql += "'{}', ".format(child.text)
            elif data_types[child.tag] == 'int':
                insert_sql += "{}, ".format(int(child.text))
        insert_sql = insert_sql[:-2] + ");"
        receiver_cursor.execute(insert_sql)
    
    # Commit changes to the receiver database
    receiver_conn.commit()

# Close the receiver database connection
receiver_conn.close()


# The user can now extract based on the receiver database

In [8]:
# Coonect to the receiver database
receiver_conn = sqlite3.connect('receiver.db')
receiver_cursor = receiver_conn.cursor()

#Execute a select query
receiver_cursor.execute('SELECT * FROM EMPLOYEE_DETAILS')
rows = receiver_cursor.fetchall()

#Process in query results
for row in rows:
    #print the values in each row
    print(row)
    
#Close the receiver database connection
receiver_cursor.close()
receiver_conn.close()

('101', 'Marshall Walker', '100000.00', '100', 'IT001')
('102', 'Arul Sharma', '250000.00', '105', 'CSE01')
('103', 'Aaron Schmidt', '150000.00', '108', 'ECE08')
('104', 'William Johansson', '225000.00', '109', 'ECE08')
('105', 'Kate Young', '150000.00', '110', 'ECE10')


# Now print all three tables

In [11]:
# Coonect to the receiver database
receiver_conn = sqlite3.connect('receiver.db')
receiver_cursor = receiver_conn.cursor()

#Define a list of tables names
table_names = ['EMPLOYEE_DETAILS', 'fruits', 'BANK_DETAILS']

#Loop through the table names and execute SELECT queries
for table_name in table_names:
    #Execute the SELECT query
    receiver_cursor.execute("SELECT * FROM {}".format(table_name))
    rows = receiver_cursor.fetchall()
    
    
    #Process in query results
    print("Table: {}".format(table_name))
    for row in rows:
        print(row)
    print()
    
#Close the receiver database connection
receiver_cursor.close()
receiver_conn.close()

Table: EMPLOYEE_DETAILS
('101', 'Marshall Walker', '100000.00', '100', 'IT001')
('102', 'Arul Sharma', '250000.00', '105', 'CSE01')
('103', 'Aaron Schmidt', '150000.00', '108', 'ECE08')
('104', 'William Johansson', '225000.00', '109', 'ECE08')
('105', 'Kate Young', '150000.00', '110', 'ECE10')

Table: fruits
('1000', 'Apple', '4', '133')
('1001', 'Apricot', '5', '175')
('1002', 'Avocado', '5', '182')
('1003', 'Banana', '5', '187')
('1004', 'Bilberry', '5', '160')
('1005', 'Blackberry', '4', '178')
('1006', 'Blackcurrant', '5', '102')
('1007', 'Blueberry', '6', '156')
('1008', 'Currant', '5', '194')
('1009', 'Cherry', '5', '182')
('1010', 'Cherimoya', '3', '169')
('1011', 'Clementine', '3', '165')
('1012', 'Date', '6', '112')
('1013', 'Damson', '1', '164')
('1014', 'Durian', '2', '157')
('1015', 'Eggplant', '6', '189')
('1016', 'Elderberry', '3', '189')
('1017', 'Feijoa', '2', '198')
('1018', 'Gooseberry', '2', '141')
('1019', 'Grape', '2', '101')
('1020', 'Grapefruit', '5', '199')
('10